In [13]:
import numpy as np
xy = np.random.rand(20,20)
np.where(xy > 0.5)[0]
# Get the leftmost pixel values over the threshold
n_pixels = 10
x_index = np.where(xy > 0.5)[0][:n_pixels]
y_index = np.where(xy > 0.5)[1][:n_pixels]
print(x_index)
print(y_index)
pixel_values = xy[x_index, y_index]

[0 0 0 0 0 0 0 0 0 1]
[ 0  3  4  7  8  9 10 12 18  0]


array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,
        1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,
        8,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 14, 14,
       14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19,
       19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19],
      dtype=int64)

In [11]:
np.where(xy > 0.5)[1]

array([ 1,  3,  4,  7, 11, 12, 13, 14, 15, 16, 17, 18, 19,  2,  3,  5, 14,
       15, 16, 17, 19,  2,  3,  7,  8, 11, 14, 16, 17,  0,  5,  6,  8,  9,
       11, 12, 13, 14, 19,  0,  4,  5,  6,  8, 11, 12, 14, 15, 18,  0,  1,
        6,  8,  9, 11, 13, 14, 19,  0,  3,  6,  8, 10, 11, 12, 13, 16, 19,
        1,  2,  3,  4,  7,  9, 11, 12, 14, 15, 18, 19,  2,  3,  8, 16, 17,
       18,  1,  6, 11, 12, 13, 14, 15, 17,  0,  1,  2,  3,  4,  6,  8, 11,
       12, 15, 18, 19,  1,  5,  6,  7,  9, 11, 13, 14, 15, 17, 19,  1,  2,
        4,  5,  7,  8, 10, 13, 14, 17,  2,  4,  5,  8,  9, 11, 12,  1,  2,
        3,  5,  7,  8,  9, 10, 11, 15, 16, 19,  5,  8,  9, 10, 13, 14, 19,
        0,  1,  3, 13, 14, 15, 17, 18, 19,  1,  5,  6,  9, 13, 14, 15, 16,
       19,  1,  2,  4,  5,  7,  8,  9, 10, 11, 12, 14, 15, 16, 17, 18,  0,
        1,  3,  5,  7,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
      dtype=int64)

In [ ]:
import numpy as np
import tofpipe
import matplotlib.pyplot as plt
import time


FREQ0 = 80320000
temp0 = 20
FREQ1 = 60240000
temp1 = 20

# The amount of pixels for a single image
# the library also defines two constants:
#  tofpipe.HVGA and tofpipe.VGA

FRAME_LEN = tofpipe.HVGA

# only needs to be constructed once with params
pipeline_2 = tofpipe.SingleAlgorithm(FREQ0, frame_len=FRAME_LEN)

# use the follwing for 2 frequency (8 phase) algorithm:
# pipeline = tofpipe.DualAlgorithm(FREQ0, FREQ1, frame_len=FRAME_LEN)

# Setup Camera

start_time = time.time()
camera = tofpipe.Camera()
camera.open()
camera.current_usecase = camera.usecases[0] #4 frames, 80.32MHz

print('Ready to take clean plate')
# Ask for user input
input()
# Get raw frames
camera.start()
clean_plate = camera.grab_raw()
camera.stop()

while True:
    print("Taking image")
    # Get raw frames
    camera.start()
    raw_frames_2 = camera.grab_raw()
    camera.stop()

    pipeline = tofpipe.SingleAlgorithm(FREQ0, frame_len=FRAME_LEN)
    pipeline.run(clean_plate[1:, 1:, :].reshape(4,-1))
    amplitude_1 = pipeline.amplitude.reshape(240,640)
    phase_1 = pipeline.phase.reshape(240,640)
    pipeline = tofpipe.SingleAlgorithm(FREQ0, frame_len=FRAME_LEN)
    pipeline.run(raw_frames_2[1:, 1:, :].reshape(4,-1))
    amplitude = pipeline.amplitude.reshape(240,640)
    phase = pipeline.phase.reshape(240,640)
    print('min phase', np.min(phase))
    print('max phase', np.max(phase))

    car_cutoff = 100
    car_silhouette = np.abs(amplitude_1 - amplitude) > car_cutoff

    # Calculate the width and height of the silhouette
    # minimum index of a 1
    if np.sum(car_silhouette) > 0:
        
        # Get the dimension of the bounding box of the silhoutte
        min_width = np.min(np.where(car_silhouette == 1)[0])
        max_width = np.max(np.where(car_silhouette == 1)[0])
        min_height = np.min(np.where(car_silhouette == 1)[1])
        max_height = np.max(np.where(car_silhouette == 1)[1])
        width = max_width - min_width
        height = max_height - min_height
        print("Width: {}, Height: {}".format(width, height))
        
        # Get the average distance for the outermost couple of pixels to the left and right.
        n_pixels = min(10, np.sum(car_silhouette))
        average_left_phase = np.mean(phase[np.where(car_silhouette == 1)[0][:n_pixels], np.where(car_silhouette == 1)[1][:n_pixels]])
        average_right_phase = np.mean(phase[np.where(car_silhouette == 1)[0][-n_pixels:], np.where(car_silhouette == 1)[1][-n_pixels:]])
        print('average_left_phase', average_left_phase)
        print('average_right_phase', average_right_phase)
        average_left_distance = average_left_phase
        average_right_phase = average_right_phase
    time.sleep(1)